#### Classification
This notebook applies a Random Forest classification model to delineate wetland extent using the preprocessed raster data.


In [1]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio     
import numpy as np  
import pandas as pd
import math
import seaborn as sns

from collections import Counter
from matplotlib.colors import ListedColormap, BoundaryNorm
from rasterio.transform import rowcol
from rasterio.windows import from_bounds
from rasterio.warp import reproject, Resampling

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.inspection import permutation_importance

from skimage.morphology import opening, square
from scipy.interpolate import griddata


In [2]:
# Base directory where data is stored
base_dir = r"C:\Users\Ethel Ogallo\Documents\ZFL1\Data"
raster_dir = r"C:\Users\Ethel Ogallo\Documents\ZFL1\Data\processed_data\cropped_rasters"

# Verify the directory exists
if not os.path.exists(base_dir):
    print(f"WARNING: Base directory does not exist: {base_dir}\nPlease check your path!")
else:
    print(f"Base directory found: {base_dir}")


Base directory found: C:\Users\Ethel Ogallo\Documents\ZFL1\Data


#### Loading Training Points
Importing and preparing the training point data for use in classification. 

In [3]:
# Define AOI extents
# Central point in decimal degrees
center_lat = 0.6
center_lon = 36.0667

# Define a bounding box: ±0.3° latitude (north-south), ±0.15° longitude (east-west)
lat_buffer = 0.3
lon_buffer = 0.15

# Compute extent (bounding box)
extents = (
    center_lon - lon_buffer,  # xmin (left)
    center_lat - lat_buffer,  # ymin (bottom)
    center_lon + lon_buffer,  # xmax (right)
    center_lat + lat_buffer   # ymax (top)
)

print("Computed extent:", extents)

# Load training points and reproject to match LST raster CRS if needed
gdf = gpd.read_file(os.path.join(base_dir, 'baringo_training_points.shp'))
with rasterio.open(os.path.join(base_dir, "input/DEM", "DEM.tif")) as src:
    dem_crs = src.crs
if gdf.crs != dem_crs:
    gdf = gdf.to_crs(dem_crs)

#gdf.head()
print(gdf)

Computed extent: (35.9167, 0.3, 36.216699999999996, 0.8999999999999999)
    id    class                  geometry
0    1    water  POINT (36.06711 0.56802)
1    2    water  POINT (36.04937 0.67043)
2    3    water  POINT (36.09725 0.47869)
3    4    water  POINT (36.09869 0.47104)
4    5    water  POINT (36.07187 0.34713)
..  ..      ...                       ...
70  70  wetland  POINT (36.08588 0.46504)
71  71  wetland  POINT (36.08948 0.46191)
72  72  wetland  POINT (36.09104 0.46122)
73  73  wetland  POINT (36.08973 0.49504)
74  74  wetland  POINT (36.07059 0.50154)

[75 rows x 3 columns]


#### Stacking All Raster Data
Combining all individual raster layers into a single multi-band raster stack, ensuring all layers are properly aligned and clipped to the area of interest (AOI)

In [ ]:
temporal_names = ["LST", "NDWI_SD", "NDVI_SD" "NDVI_range"]
static_names = ["relativeDEM", "TWI", "Inundation"]

years = range(2018, 2025)  # inclusive of 2024
feature_stack_by_year = {}

# Function to load + resample
def load_and_resample(path, ref_shape, ref_transform, ref_crs):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(float)
        nodata = src.nodata
        if nodata is not None:
            arr[arr == nodata] = np.nan  # convert nodata to NaN

        if src.shape != ref_shape or src.crs != ref_crs or src.transform != ref_transform:
            dst = np.empty(ref_shape, dtype=arr.dtype)
            reproject(
                source=arr,
                destination=dst,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.bilinear
            )
            arr = dst
    return arr

# Loop over all years
for year in years:
    print(f"Processing year {year}...")

    # Reference raster (DEM for that year)
    ref_path = os.path.join(raster_dir, str(year), "DEM.tif")
    if not os.path.exists(ref_path):
        print(f"Reference DEM missing for {year}, skipping.")
        continue

    with rasterio.open(ref_path) as ref_src:
        ref_shape = ref_src.shape
        ref_transform = ref_src.transform
        ref_crs = ref_src.crs

    layers, layer_names = [], []

    for name in temporal_names + static_names:
        path = os.path.join(raster_dir, str(year), f"{name}.tif")
        if os.path.exists(path):
            arr = load_and_resample(path, ref_shape, ref_transform, ref_crs)
            layers.append(arr)
            layer_names.append(name)
        else:
            print(f"Missing layer '{name}' for {year}")

    if layers:
        stack = np.stack(layers, axis=-1)
        feature_stack_by_year[year] = {"stack": stack, "layer_names": layer_names}
        print(f"Year {year}: Stacked {len(layers)} layers -> stack shape: {stack.shape}")
    else:
        print(f"No layers stacked for {year}.")


Processing year 2018...
Missing layer 'NDWI_SD' for 2018
Missing layer 'NDVI_SDNDVI_range' for 2018
Missing layer 'relativeDEM' for 2018
Missing layer 'Inundation_freq' for 2018
Year 2018: Stacked 2 layers -> stack shape: (2222, 1113, 2)
Processing year 2019...
Missing layer 'NDWI_SD' for 2019
Missing layer 'NDVI_SDNDVI_range' for 2019
Missing layer 'relativeDEM' for 2019
Missing layer 'Inundation_freq' for 2019
Year 2019: Stacked 2 layers -> stack shape: (2222, 1113, 2)
Processing year 2020...
Missing layer 'NDWI_SD' for 2020
Missing layer 'NDVI_SDNDVI_range' for 2020
Missing layer 'relativeDEM' for 2020
Missing layer 'Inundation_freq' for 2020
Year 2020: Stacked 2 layers -> stack shape: (2222, 1113, 2)
Processing year 2021...
Missing layer 'NDWI_SD' for 2021
Missing layer 'NDVI_SDNDVI_range' for 2021
Missing layer 'relativeDEM' for 2021
Missing layer 'Inundation_freq' for 2021
Year 2021: Stacked 2 layers -> stack shape: (2222, 1113, 2)
Processing year 2022...
Missing layer 'NDWI_SD' 

In [ ]:
temporal_names = ["LST", "NDWI_SD", "NDVI_SD", "NDWI_Range", "NDVI_range"]
static_names = ["relativeDEM", "TWI", "Inundation_v2"]

year = range(2018, 2025)
feature_stack_by_year = {}

# Reference raster
ref_path = os.path.join(raster_dir, str(year), "DEM.tif")
with rasterio.open(ref_path) as ref_src:
    ref_shape = ref_src.shape
    ref_transform = ref_src.transform
    ref_crs = ref_src.crs
    ref_meta = ref_src.meta.copy()

# Load + resample function
def load_and_resample(path, ref_shape, ref_transform, ref_crs):
    with rasterio.open(path) as src:
        arr = src.read(1)
        if src.shape != ref_shape or src.crs != ref_crs or src.transform != ref_transform:
            dst = np.empty(ref_shape, dtype=arr.dtype)
            reproject(
                source=arr,
                destination=dst,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.bilinear
            )
            arr = dst
    return arr

# Load + stack with nodata handling
# Load + stack with correct nodata handling
layers, layer_names = [], []

for name in temporal_names + static_names:
    path = os.path.join(raster_dir, str(year), f"{name}.tif")
    if os.path.exists(path):
        with rasterio.open(path) as src:
            arr = src.read(1).astype(float)
            nodata = src.nodata
            if nodata is not None:
                arr[arr == nodata] = np.nan  # convert -9999 -> NaN

        # Now resample after nodata conversion
        if arr.shape != ref_shape or src.crs != ref_crs or src.transform != ref_transform:
            dst = np.empty(ref_shape, dtype=arr.dtype)
            reproject(
                source=arr,
                destination=dst,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.bilinear
            )
            arr = dst

        layers.append(arr)
        layer_names.append(name)
    else:
        print(f"Missing layer '{name}'")


# Stack
stack = np.stack(layers, axis=-1)
feature_stack_by_year[year] = {"stack": stack, "layer_names": layer_names}
print(f"Year {year}: Stacked {len(layers)} layers -> stack shape: {stack.shape}")

In [ ]:
stack = feature_stack_by_year[year]["stack"]
layer_names = feature_stack_by_year[year]["layer_names"]

# Assign colormaps
colormap_map = {
    "LST": "inferno",
    "NDWI_SD": "Blues",
    "NDVI_SD": "Greens",
    "NDWI_Range": "PuBu",
    "NDVI_range": "YlGn",
    "relativeDEM": "terrain",
    "TWI": "cividis",
    "Inundation_v2": "Blues"
}

n_layers = len(layer_names)
n_cols = math.ceil(n_layers / 2)
n_rows = 2

fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))
axes = axes.flatten()

for i in range(n_layers):
    data = stack[:, :, i]
    
    # Mask NaN values so they don't appear
    masked_data = np.ma.masked_invalid(data)
    
    cmap = colormap_map.get(layer_names[i], "viridis")
    img = axes[i].imshow(masked_data, cmap=cmap)
    axes[i].set_title(layer_names[i], fontsize=10)
    axes[i].axis('off')
    fig.colorbar(img, ax=axes[i], fraction=0.046, pad=0.04)

# Hide extra subplots if any
for j in range(n_layers, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


##### Extracting Pixel Values at Training Points
Sampling raster feature values at the locations of training points to create the input data for model training.

In [ ]:
# Get transform of cropped DEM once, since extent is the same for all years
dem_path = os.path.join(raster_dir, "2024", "DEM.tif")
with rasterio.open(dem_path) as src:
    window = from_bounds(*extents, transform=src.transform)
    cropped_transform = src.window_transform(window)

# Prepare list of (x, y) coordinates from the training points GeoDataFrame
coords = [(geom.x, geom.y) for geom in gdf.geometry]

# Initialize dict 
if 'training_data_by_year' not in globals():
    training_data_by_year = {}

for year, data in feature_stack_by_year.items():
    feature_stack = data['stack']
    transform = cropped_transform

    # Convert (x, y) coordinates to raster (row, col)
    rows_cols = [rowcol(transform, x, y) for x, y in coords]

    pixels, labels = [], []
    for (row, col), label in zip(rows_cols, gdf["class"]):
        # Check bounds
        if 0 <= row < feature_stack.shape[0] and 0 <= col < feature_stack.shape[1]:
            px_values = feature_stack[row, col, :]
            # Ignore if any value is NaN
            if not np.any(np.isnan(px_values)):
                pixels.append(px_values)
                labels.append(label)

    # Convert to numpy arrays
    X = np.array(pixels)
    y = np.array(labels)

    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Store
    training_data_by_year[year] = {
        "X": X,
        "y": y_encoded,
        "label_encoder": le
    }

    print(f"Year {year}: extracted {len(y)} samples with {X.shape[1]} features")


In [ ]:
## for 2024 only

# Prepare list of (x, y) coordinates from the training points GeoDataFrame
coords = [(geom.x, geom.y) for geom in gdf.geometry]

# Initialize dict 
if 'training_data_by_year' not in globals():
    training_data_by_year = {}

for year, data in feature_stack_by_year.items():
    feature_stack = data['stack']
    transform = ref_transform  

    pixels, labels = [], []
    for (x, y), label in zip(coords, gdf["class"]):
        row, col = rowcol(transform, x, y)
        # Check bounds
        if 0 <= row < feature_stack.shape[0] and 0 <= col < feature_stack.shape[1]:
            px_values = feature_stack[row, col, :]
            # Ignore if any value is NaN
            if not np.any(np.isnan(px_values)):
                pixels.append(px_values)
                labels.append(label)

    # Convert to numpy arrays
    X = np.array(pixels)
    y = np.array(labels)

    if len(X) == 0:
        print(f"Year {year}: No valid training points inside raster!")
        continue

    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Store
    training_data_by_year[year] = {
        "X": X,
        "y": y_encoded,
        "label_encoder": le
    }

    print(f"Year {year}: extracted {len(y)} samples with {X.shape[1]} features")


In [ ]:
year = 2024
df_samples = pd.DataFrame(training_data_by_year[year]['X'], columns=layer_names)
df_samples['class'] = training_data_by_year[year]['y']
df_samples.iloc[:-10]
#df_samples.shape

In [ ]:
class_counts = df_samples['class'].value_counts()
print("Number of training points per class:")
print(class_counts)

#### Stratified 5-Fold Cross-Validation with Random Forest
This section covers training the Random Forest classifier using the prepared training dataset.
The stratified 5-fold cross-validation is applied to evaluate model performance, ensuring each fold preserves the class distribution.

The workflow includes both combined train/test splitting and cross-validation to optimize and validate the classification model. 


In [ ]:
def train_model_per_year(year, training_data, test_size=0.2, n_estimators_cv=100, n_estimators_final=200, random_state=42):
    """
    Train Random Forest model for each year with train/test split and cross-validation.
    
    Returns:
        model: final trained model on combined train+val
        metrics: dict with CV and test accuracies and predictions
    """
    data = training_data[year]
    X, y, le = data["X"], data["y"], data["label_encoder"]

    from collections import Counter
    class_counts = Counter(y)
    min_class_count = min(class_counts.values())

    if min_class_count < 2:
        print(f"Warning: Year {year} has class with less than 2 samples. Skipping stratified split and CV.")

        # Train final model on all data (no split)
        final_rf = RandomForestClassifier(n_estimators=n_estimators_final, random_state=random_state)
        final_rf.fit(X, y)

        metrics = {
            "cv_true": None,
            "cv_pred": None,
            "cv_accuracy": None,
            "y_test": None,
            "y_test_pred": None,
            "test_accuracy": None,
            "label_encoder": le,
        }
        return final_rf, metrics

    # Proceed with stratified split since all classes have >= 2 samples
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # Cross-validation training
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    cv_true, cv_pred = [], []
    for train_idx, val_idx in skf.split(X_train_val, y_train_val):
        rf = RandomForestClassifier(n_estimators=n_estimators_cv, random_state=random_state)
        rf.fit(X_train_val[train_idx], y_train_val[train_idx])
        y_val_pred = rf.predict(X_train_val[val_idx])
        cv_true.extend(y_train_val[val_idx])
        cv_pred.extend(y_val_pred)

    cv_accuracy = accuracy_score(cv_true, cv_pred)

    # Final model on combined train+val
    final_rf = RandomForestClassifier(n_estimators=n_estimators_final, random_state=random_state)
    final_rf.fit(X_train_val, y_train_val)

    # Test set evaluation
    y_test_pred = final_rf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    metrics = {
        "cv_true": cv_true,
        "cv_pred": cv_pred,
        "cv_accuracy": cv_accuracy,
        "y_test": y_test,
        "y_test_pred": y_test_pred,
        "test_accuracy": test_accuracy,
        "label_encoder": le,
    }

    return final_rf, metrics



Using the trained Random Forest model, the wetland classes on the study area is predicted.
Following classification, the area covered by each wetland class is estimated in hectares (Ha).

In [ ]:
# Train model for a specific year
def prediction_per_year(year, model, feature_stack_by_year, extent, label_encoder):
    """
    Predict classification raster for one year and compute area per class.
    
    Returns:
        pred_img: 2D numpy array of predictions with -1 outside AOI
        class_areas: dict of class names to area in hectares
    """
    feature_stack = feature_stack_by_year[year]["stack"]
    flat = feature_stack.reshape(-1, feature_stack.shape[-1])
    valid_mask = ~np.isnan(flat).any(axis=1)

    pred = np.full(flat.shape[0], -1, dtype=int)
    pred[valid_mask] = model.predict(flat[valid_mask])
    pred_img = pred.reshape(feature_stack.shape[:2])

    counts = Counter(pred_img[pred_img >= 0])

    # Calculate pixel area in hectares
    lat = (extent[1] + extent[3]) / 2
    deg_to_m = 111320  # meters per degree
    pixel_w = abs(extent[2] - extent[0]) / pred_img.shape[1]
    pixel_h = abs(extent[3] - extent[1]) / pred_img.shape[0]
    pixel_area_ha = (pixel_w * deg_to_m * math.cos(math.radians(lat))) * (pixel_h * deg_to_m) / 10_000

    class_areas = {
        label_encoder.inverse_transform([cls])[0]: count * pixel_area_ha for cls, count in counts.items()
    }

    return pred_img, class_areas

In [ ]:
models_by_year = {}
metrics_by_year = {}
predictions_by_year = {}

for year in sorted(training_data_by_year.keys()):
    print(f"\nProcessing year {year}...")

    # Train model
    model, metrics = train_model_per_year(year, training_data_by_year)
    models_by_year[year] = model
    metrics_by_year[year] = metrics

    print(f"Year {year} CV Accuracy: {metrics['cv_accuracy']:.4f}")
    print(f"Year {year} Test Accuracy: {metrics['test_accuracy']:.4f}")

    # Predict
    pred_img, class_areas = prediction_per_year(year, model, feature_stack_by_year, extent, metrics["label_encoder"])
    predictions_by_year[year] = {
        "pred_img": pred_img,
        "class_areas": class_areas,
    }

    print(f"Prediction complete for year {year}.")          

In [ ]:
year = 2024
models_by_year = {}
metrics_by_year = {}
predictions_by_year = {}

print(f"\nProcessing year {year}...")

# Train model
model, metrics = train_model_per_year(year, training_data_by_year)
models_by_year[year] = model
metrics_by_year[year] = metrics

print(f"Year {year} CV Accuracy: {metrics['cv_accuracy']:.4f}")
print(f"Year {year} Test Accuracy: {metrics['test_accuracy']:.4f}")

# Predict
pred_img, class_areas = prediction_per_year(
    year, 
    model, 
    feature_stack_by_year, 
    extents, 
    metrics["label_encoder"]
)
predictions_by_year[year] = {
    "pred_img": pred_img,
    "class_areas": class_areas,
}

print(f"Prediction complete for year {year}.")


#### Confusion Matrix
Evaluating classification accuracy by comparing predicted classes against true labels using a confusion matrix.

In [ ]:
year = 2024  # change as needed

# The trained RF model
rf_model = models_by_year[year]

# Define feature names (matching the actual feature stack order)
features = ['LST', 'NDWI_SD','NDWI_Range', 'NDVI_SD','NDVI_range', 'TWI','Inundation_v2', 'relativeDEM'] 

importances = (rf_model.feature_importances_) * 100
idx = importances.argsort()[::-1]

plt.figure(figsize=(7, 4))
plt.bar(np.array(features)[idx], importances[idx], color='steelblue')
plt.xticks(rotation=45, ha='right')
plt.title(f"Feature Importances - {year}")
plt.ylabel("Importance (%)")
plt.xlabel("Feature")
plt.tight_layout()
plt.show()

# model test results metrics
le = metrics_by_year[year]['label_encoder']
y_test = metrics_by_year[year]['y_test']
y_test_pred = metrics_by_year[year]['y_test_pred']

print(f"\n=== {year} Test set Classification Report ===")
print(classification_report(y_test, y_test_pred, target_names=le.classes_))

# Plot confusion matrix
#cv_true = metrics_by_year[year]['cv_true']
#cv_pred = metrics_by_year[year]['cv_pred']
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(4, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix for test set - {year}')
plt.tight_layout()
plt.show()

print(f"\n=== Estimated area per class for year {year}:")
for cls, area in predictions_by_year[year]['class_areas'].items():
    print(f"  {cls}: {area:.2f} ha")


#### Visualization of Yearly classification
Displaying the predicted wetland classification results for each year.

In [ ]:
years = sorted(predictions_by_year.keys())  
n_years = len(years)

# Setup colormap and norm 
le = metrics_by_year[years[0]]['label_encoder']
labels = le.classes_
cmap = ListedColormap(['#145a32', '#abb2b9', '#3498db', '#f39c12'][:len(labels)])
norm = BoundaryNorm(np.arange(len(labels)+1)-0.5, cmap.N)
xmin, ymin, xmax, ymax = extents


# Define subplot grid size (2 rows, adjust columns)
nrows = 2
ncols = (n_years + 1) // 2

fig, axes = plt.subplots(nrows, ncols, figsize=(6*ncols, 7*nrows))
axes = axes.flatten()

for ax, year in zip(axes, years):
    pred_img = predictions_by_year[year]['pred_img']

    im = ax.imshow(pred_img, cmap=cmap, norm=norm, 
                   extent=[xmin, xmax, ymin, ymax], origin='upper')
    ax.set_title(f"Classification Prediction for {year}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_aspect('equal')

    # Add colorbar per subplot
    cbar = fig.colorbar(im, ax=ax, ticks=np.arange(len(labels)))
    cbar.ax.set_yticklabels(labels)
    cbar.set_label("Class")

# Remove empty subplots if any
for ax in axes[n_years:]:
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
year = 2024
pred_img = predictions_by_year[year]['pred_img']


In [ ]:
from rasterio.transform import array_bounds

fig, ax = plt.subplots(figsize=(8, 7))

# Setup colormap and norm 
le = metrics_by_year[year]['label_encoder']
labels = le.classes_
cmap = ListedColormap(['#145a32', '#abb2b9', '#3498db', '#f39c12'][:len(labels)])
norm = BoundaryNorm(np.arange(len(labels)+1)-0.5, cmap.N)

# Compute extent from raster shape and ref_transform
height, width = pred_img.shape
xmin, ymin, xmax, ymax = array_bounds(height, width, ref_transform)
extent = [xmin, xmax, ymin, ymax]

im = ax.imshow(pred_img, cmap=cmap, norm=norm, 
               extent=extent, origin='upper')
ax.set_title(f"Classification Prediction for {year}")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_aspect('equal')

cbar = fig.colorbar(im, ax=ax, ticks=np.arange(len(labels)))
cbar.ax.set_yticklabels(labels)
cbar.set_label("Class")

plt.tight_layout()
plt.show()


In [ ]:
# save the predictions for the year 2024
out_tif = f"C:\\Users\\Ethel Ogallo\\Documents\\ZFL1\\Data\\classification_{year}.tif"
 

with rasterio.open(
    out_tif, 'w', driver='GTiff',
    height=pred_img.shape[0], width=pred_img.shape[1],
    count=1, dtype=pred_img.dtype,
    crs=dem_crs, transform=cropped_transform
) as dst:
    dst.write(pred_img, 1)


#### CART

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_text, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Get training data for 2024
X = training_data_by_year[2024]["X"]
y = training_data_by_year[2024]["y"]
le = training_data_by_year[2024]["label_encoder"]


# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# CART model
cart = DecisionTreeClassifier(
    criterion='gini',
    max_depth=None,
    min_samples_leaf=1,
    #class_weight='balanced',  
    random_state=42
)

#cart =DecisionTreeClassifier()

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(cart, X_train, y_train, cv=cv, scoring='accuracy')
print(f"5-Fold CV Accuracy: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.3f}")

# Train on full training set
cart.fit(X_train, y_train)
y_pred = cart.predict(X_test)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap='Blues', values_format='d')
plt.title("CART Confusion Matrix - 2024")
plt.show()

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# feature importance
importances = cart.feature_importances_
layer_names = feature_stack_by_year[2024]["layer_names"]
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(6,6))
plt.barh(np.array(layer_names)[indices], importances[indices], color='steelblue')
plt.gca().invert_yaxis()
plt.title("CART Feature Importance - 2024")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# Optional: print tree structure
print(export_text(cart, feature_names=layer_names))


In [ ]:
from rasterio.transform import array_bounds

# Year
year = 2024

# Get the stack and transform
feature_stack = feature_stack_by_year[year]['stack']
layer_names = feature_stack_by_year[year]['layer_names']
n_rows, n_cols, n_features = feature_stack.shape

# Flatten for prediction
flat_pixels = feature_stack.reshape(-1, n_features)

# Mask NaNs
mask = ~np.any(np.isnan(flat_pixels), axis=1)

# Predict using CART
flat_pred = np.full(flat_pixels.shape[0], fill_value=-1, dtype=int)
flat_pred[mask] = cart.predict(flat_pixels[mask])

# Ensure predictions match LabelEncoder indices
le = training_data_by_year[year]['label_encoder']
flat_pred_corrected = np.full_like(flat_pred, -1)
flat_pred_corrected[mask] = le.transform(le.inverse_transform(flat_pred[mask]))

# Reshape to raster
pred_img = flat_pred_corrected.reshape(n_rows, n_cols)

# Build colormap tied to class labels
class_colors = {
    'forest': '#145a32',     # dark green
    'rangeland': '#abb2b9',  # gray
    'water': '#3498db',      # blue
    'wetland': '#f39c12'     # orange
}
cmap_list = [class_colors[cls] for cls in le.classes_]
cmap = ListedColormap(cmap_list)
norm = BoundaryNorm(np.arange(len(le.classes_)+1)-0.5, cmap.N)

# Compute extent properly
xmin, ymin, xmax, ymax = array_bounds(n_rows, n_cols, ref_transform)
extent = [xmin, xmax, ymin, ymax]

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(pred_img, cmap=cmap, norm=norm, extent=extent, origin='upper')
cbar = fig.colorbar(im, ax=ax, ticks=np.arange(len(le.classes_)))
cbar.ax.set_yticklabels(le.classes_)
cbar.set_label("Class")
ax.set_title(f"Wetland Extent CART")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_aspect('equal')
plt.tight_layout()
plt.show()


In [ ]:
from rasterio.features import shapes
from shapely.geometry import shape
from shapely.ops import unary_union
from scipy.ndimage import binary_dilation

# Identify pixels of interest
aoi_classes = ['water', 'wetland']
aoi_indices = le.transform(aoi_classes)
aoi_mask = np.isin(pred_img, aoi_indices)

# using dilation to merge nearby patches: includes small non-water/wetland gaps inside the main AOI
dilated_mask = binary_dilation(aoi_mask, iterations=6)  

# Extract polygons
polygons_generator = shapes(dilated_mask.astype(np.uint8), mask=dilated_mask, transform=ref_transform)
polygons = [shape(geom) for geom, value in polygons_generator if value == 1]

# Merge polygons into one main AOI
aoi_gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")

# Save AOI
aoi_gdf.to_file(os.path.join(base_dir, 'wetland_extent.shp'))


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from rasterio.transform import array_bounds

# Build colormap for CART prediction
class_colors = {
    'forest': '#145a32',     # dark green
    'rangeland': '#abb2b9',  # gray
    'water': '#3498db',      # blue
    'wetland': '#f39c12'     # orange
}
cmap_list = [class_colors[cls] for cls in le.classes_]
cmap = ListedColormap(cmap_list)
norm = BoundaryNorm(np.arange(len(le.classes_)+1)-0.5, cmap.N)

# Compute raster extent
xmin, ymin, xmax, ymax = array_bounds(n_rows, n_cols, ref_transform)
extent = [xmin, xmax, ymin, ymax]

# Plot
fig, ax = plt.subplots(figsize=(8, 6))

# Classification raster
im = ax.imshow(pred_img, cmap=cmap, norm=norm, extent=extent, origin='upper')

# Overlay AOI polygons
aoi_gdf2 = gpd.read_file(os.path.join(base_dir, 'wetland_extent_2024.shp'))
aoi_gdf2.boundary.plot(ax=ax, color='red', linewidth=1)

# Colorbar
cbar = fig.colorbar(im, ax=ax, ticks=np.arange(len(le.classes_)))
cbar.ax.set_yticklabels(le.classes_)
cbar.set_label("Class")

ax.set_title(f"Extent (water + wetland)")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_aspect('equal')

plt.tight_layout()
plt.show()
